# Chương 9: Không gian vector (I)
File enwiki-20150112-400-r10-105752.txt.bz2 là file nén dạng bzip2 của 105752 file text được lấy mẫu ngẫu nhiên (tỷ lệ 1/10) từ các bài báo trên Wikipedia có trên 400 từ. Các bài báo trên Wikipedia được lấy vào ngày 12 tháng 1 năm 2015. Sử dụng dữ liệu file này làm corpus để học các vector thể hiện ý nghĩa của các từ. Trong nửa đầu của chương 9, bạn được yêu cầu trích xuất các context của các từ, trích xuất đặc trưng, và dùng phương pháp PCA để giảm bớt số chiều của dữ liệu. Nửa sau của chương 9 yêu cầu bạn tính độ tương tự của các từ sử dụng các word vectors đã học từ corpus.

Chú ý, bài 83 yêu cầu 7GB memory. Trong trường hợp lượng memory của bạn không đủ, bạn cần có các phương pháp xử lý thích hợp hoặc sử dụng sample 1/100 của dữ liệu trong file enwiki-20150112-400-r10-105752.txt.bz2.

# Download data

In [4]:
!wget http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2

--2020-04-18 15:55:04--  http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2
Resolving www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)... 130.34.192.83
Connecting to www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)|130.34.192.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221903910 (212M) [application/x-bzip2]
Saving to: ‘enwiki-20150112-400-r10-105752.txt.bz2’

enwiki-20150112-400 100%[===================>] 211.62M  11.2MB/s    in 19s     

2020-04-18 15:55:24 (11.1 MB/s) - ‘enwiki-20150112-400-r10-105752.txt.bz2’ saved [221903910/221903910]



In [0]:
!bzip2 -d enwiki-20150112-400-r10-105752.txt.bz2

# 80. Tiền xử lý dữ liệu
Sử dụng khoảng trắng là ký tự ngăn cách để tokenize các từ trong các câu. Phương pháp này có nhược điểm là các từ thu được sẽ còn các ký tự đặc biệt như dấu câu, hoặc dấu ngoặc. Vì thế sau khi tokenize các từ trong corpus, tiến hành các xử lý sau đây.

* Xoá các ký tự đặc biệt xuất hiện ở đầu và cuối các từ: .,!?;:()[]'"
* Xoá các từ chỉ gồm ký tự trắng
Sau khi tiền xử lý dữ liệu, lưu file dữ liệu gồm danh sách các từ cách nhau bởi khoảng trắng.